### FE545 - Homework #3

**Author**: Sid Bhatia

**Date**: April 6th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Steve Yang

#### Problem 1

The **trinomial** option pricing model is an option pricing model incorporating three possible values that an underlying asset can have in one time period. The three possible values the underlying asset can have in a time period may be greater than, the same as, or less than the current value. The trinomial model uses an iterative procedure, allowing for the specification of nodes, or points in time, during the time span between the valuation date and the option’s expiration date.

The trinomial option pricing model, proposed by *Phelim Boyle* in 1986, is considered to be more accurate than the binomial model, and will compute the same results, but in fewer steps. However, the trinomial model has not gained as much popularity as the other models.

The trinomial option pricing model differs from the binomial option pricing model in one key aspect by incorporating another possible value in one time period. Under the binomial option pricing model, it is assumed that the value of the underlying asset will either be greater than or less than, its current value.

The trinomial model, on the other hand, incorporates a third possible value, which incorporates a zero change in value over a time period. This assumption makes the trinomial model more relevant to real life situations, as it is possible that the value of an underlying asset may not change over a time period, such as a month or a year.

Under the trinomial method, the underlying stock price is modeled as a recombining tree, where, at each node the price has **three possible paths**: an up, down and stable or middle path.

We will use a trinomial tree model defined by

$$
S(t + \Delta t) = \begin{cases} 

S(t)u & \text{w.p} & p_u \\
S(t) & \text{w.p} & 1 - p_u - p_d \\ \tag{1}
S(t)d & \text{w.p} & p_d

\end{cases}
$$

$$
\\[20pt]
$$

$$
\mathbb{E}[S(t_{i + 1}) \mid S(t_i)] = e^{r\Delta t}S(t_i)

\\[5pt] \tag{2}

\mathbb{V}[S(t_{i + 1}) \mid S(t_i)] = \Delta t S^2(t_i) \sigma^2 + \mathcal{O}(\Delta t) 
$$

The expectation & variance (*Condition 2*) are/is a **standard equilibrium** or **no arbitrage condition**: it states that the average return from the asset should be equal to the risk free-return. It can be re-written in the following explicit form:

$$
1 - p_u - p_d + p_u u + p_d d = e^{r \Delta t}
$$

Conditions (2, 4) impose two constraints on 4 parameters of the tree. An extra constraint comes from the requirement that the size of the upward jump is the reciprocal of the size of the downward jump, $$ud = 1 \tag{3}$$

While  this condition is not always used for a trinomial tree construction, it greatly simplifies the complexity of the numerical scheme: it leads to a **recombining tree**, which has the number of nodes growing *polynomially* with the number of levels rather than *exponentially*.

Given the knowledge of **jump sizes** $u, d$ and the **transition probabilities** $p_u, p_d$, it is now possible to find the value of the underlying asset $S$ for any sequence of price movements. A *directed graph* with nodes labeled by asset prices and edges connecting nodes s separated by one time step and a single price jump is the desired trinomial pricing tree, see **Figure 1**:

<figure>
    <img src="./fe545-hw3-figures/fe545-hw3-figure1.png" alt="Trinomial Tree" style="width: 50%; display: block; margin-left: auto; margin-right: auto;">
    <figcaption style="text-align: center; font-size: larger; margin-top: 10px">Figure 1: Trinomial Tree</figcaption>
</figure>